# Template

In [ ]:
import numpy as np
import time
from datetime import datetime, timedelta
from neuroplatform import StimShape, StimParam, IntanSoftware, Trigger, Database, StimPolarity, Experiment

token = "XSALK6J9C4" 
exp = Experiment(token)
print(f'Electrodes: {exp.electrodes}')  # Electrodes that you can use

# Configure stimulation parameters for each electrode
stim_params = []
for index in exp.electrodes:
    stim_param = StimParam()
    stim_param.index = index  # Set to current electrode in the loop

    # Configure unique trigger key for each electrode, assuming it's mapped to the index
    stim_param.trigger_key = index

    # Setting common stimulation parameters
    stim_param.phase_duration1 = 200.0
    stim_param.phase_duration2 = 200.0
    stim_param.phase_amplitude1 = 8.0
    stim_param.phase_amplitude2 = 8.0
    stim_param.stim_shape = StimShape.Biphasic
    stim_param.polarity = StimPolarity.PositiveFirst
    stim_param.enable_amp_settle = True
    stim_param.pre_stim_amp_settle = 0.0
    stim_param.post_stim_amp_settle = 1000.0
    stim_param.enable_charge_recovery = True
    stim_param.post_charge_recovery_on = 0.0
    stim_param.post_charge_recovery_off = 100.0
    stim_param.interphase_delay = 0.0

    stim_params.append(stim_param)

intan = IntanSoftware()
trigger_gen = Trigger()

try:
    if exp.start():  # Signal the start of an experiment to all users
        # Measure impedance
        intan.impedance()

        # Disable Variation STD (keep a fixed threshold)
        intan.var_threshold(False)

        # Send stim parameters
        intan.send_stimparam(stim_params)

        # Manually send trigger signals
        for stim_param in stim_params:
            trigger = np.zeros(16, dtype=np.uint8)
            trigger[stim_param.trigger_key] = 1

            for _ in range(40):  # Send trigger signal 40 times for each electrode
                time.sleep(5)  # Wait 5 seconds between each trigger
                trigger_gen.send(trigger)

        # Disable all stims
        for stim in stim_params:
            stim.enable = False
        intan.send_stimparam(stim_params)

finally:
    # Close the connection to trigger generator
    trigger_gen.close()
    # Enable variation threshold again
    intan.var_threshold(True)
    # Close the connection to intan software
    intan.close()
    # Signal the end of an experiment to all users
    exp.stop()